In [1]:
import requests

In [2]:
BSCSCAN_KEY = "28MFHGSYR31QX9TEFIGCRGN3PGV24B1YBV"
CONTRACT_ADDRESS = "0x45c54210128a065de780C4B0Df3d16664f7f859e"
DEPLOYMENT_BLOCK = 26098112

In [3]:
def fetch_logs(parameters,address=CONTRACT_ADDRESS,
            from_block=DEPLOYMENT_BLOCK,to_block="latest"):
    # make GET request to bscscan

    print(f"Getting logs from block {from_block} to {to_block}")

    req_params = {
        "apikey":BSCSCAN_KEY,
        "module":"logs",
        "action":"getLogs",
        "address":address,
        "fromBlock":from_block,
        "toBlock":to_block,
    }
    req_params.update(parameters)

    response = requests.get("https://api.bscscan.com/api",
                            params=req_params)
    
    

    json_response = response.json()

    if json_response['message'] != "OK":
        return False, json_response['message']

    # check the number of results
    if len(json_response['result']) >= 1000:
        # get the last block number
        last_block_hex = json_response['result'][-1]['blockNumber']
        last_block = int(last_block_hex,16)
        # call the function again with the last block number
        ok, new_results = fetch_logs(parameters,address,last_block,
                to_block=to_block)

        if not ok:
            return True, json_response['result']
        
        return True, json_response['result'] + new_results
        
    return True, json_response['result']

In [4]:
logs = fetch_logs({}, to_block="26098112", from_block=25098112)

Getting logs from block 25098112 to 26098112
Getting logs from block 25105273 to 26098112
Getting logs from block 25113902 to 26098112
Getting logs from block 25121610 to 26098112
Getting logs from block 25127524 to 26098112
Getting logs from block 25134946 to 26098112
Getting logs from block 25144779 to 26098112
Getting logs from block 25151496 to 26098112
Getting logs from block 25157192 to 26098112
Getting logs from block 25164545 to 26098112
Getting logs from block 25172847 to 26098112
Getting logs from block 25178733 to 26098112
Getting logs from block 25184466 to 26098112
Getting logs from block 25192937 to 26098112
Getting logs from block 25201671 to 26098112
Getting logs from block 25207426 to 26098112
Getting logs from block 25211832 to 26098112
Getting logs from block 25217321 to 26098112
Getting logs from block 25224651 to 26098112
Getting logs from block 25231693 to 26098112
Getting logs from block 25236899 to 26098112
Getting logs from block 25241462 to 26098112
Getting lo

In [5]:
from typing import Any, Dict, cast, Union

from eth_typing import HexStr
from eth_utils import event_abi_to_log_topic
from hexbytes import HexBytes
from web3._utils.abi import get_abi_input_names, get_abi_input_types, map_abi_data
from web3._utils.normalizers import BASE_RETURN_NORMALIZERS
from web3.contract import Contract


class EventLogDecoder:
    def __init__(self, contract: Contract):
        self.contract = contract
        self.event_abis = [abi for abi in self.contract.abi if abi['type'] == 'event']
        self._sign_abis = {event_abi_to_log_topic(abi): abi for abi in self.event_abis}
        self._name_abis = {abi['name']: abi for abi in self.event_abis}

    def decode_log(self, result: Dict[str, Any]):

        data = [t[2:] for t in result['topics']]
        data += [result['data'][2:]]
        data = "0x" + "".join(data)
        return self.decode_event_input(data)

    def decode_event_input(self, data: Union[HexStr, str], name: str = None) -> Dict[str, Any]:
        # type ignored b/c expects data arg to be HexBytes
        data = HexBytes(data)  # type: ignore
        selector, params = data[:32], data[32:]

        if name:
            func_abi = self._get_event_abi_by_name(event_name=name)
        else:
            func_abi = self._get_event_abi_by_selector(selector)

        event_name = func_abi['name']

        names = get_abi_input_names(func_abi)
        types = get_abi_input_types(func_abi)

        decoded = self.contract.web3.codec.decode(types, cast(HexBytes, params))
        normalized = map_abi_data(BASE_RETURN_NORMALIZERS, types, decoded)

        return event_name, dict(zip(names, normalized))

    def _get_event_abi_by_selector(self, selector: HexBytes) -> Dict[str, Any]:
        try:
            return self._sign_abis[selector]
        except KeyError:
            raise ValueError("Event is not presented in contract ABI.")

    def _get_event_abi_by_name(self, event_name: str) -> Dict[str, Any]:
        try:
            return self._name_abis[event_name]
        except KeyError:
            raise KeyError(f"Event named '{event_name}' was not found in contract ABI.")

In [6]:
from web3 import Web3
import json

with open("abi.json") as f:
    abi = json.load(f)

w3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))

In [7]:
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=abi)
eld = EventLogDecoder(contract)

decoded_logs = []
for log in logs[1]:
    event_name, event_data = eld.decode_log(log)
    new_log = log.copy()
    new_log['event_name'] = event_name
    del new_log['topics']
    new_log['blockNumber'] = int(new_log['blockNumber'], 16)
    del new_log['blockHash']
    new_log['timeStamp'] = int(new_log['timeStamp'], 16)
    new_log['gasPrice'] = int(new_log['gasPrice'], 16)
    new_log['gasUsed'] = int(new_log['gasUsed'], 16)
    new_log['logIndex'] = int(new_log['logIndex'], 16)
    del new_log['transactionHash']
    del new_log['address']
    del new_log['data']
    new_log.update(event_data)
    decoded_logs.append(new_log)

decoded_logs

[{'blockNumber': 25098115,
  'timeStamp': 1674667822,
  'gasPrice': 5000000000,
  'gasUsed': 203064,
  'logIndex': 124,
  'transactionIndex': '0x47',
  'event_name': 'Harvest',
  'sender': '0x3Fef2A25C2622cD0d0b57690479D7350c232038C',
  'amount': 168328969601476841510},
 {'blockNumber': 25098115,
  'timeStamp': 1674667822,
  'gasPrice': 5000000000,
  'gasUsed': 203064,
  'logIndex': 128,
  'transactionIndex': '0x47',
  'event_name': 'Withdraw',
  'sender': '0x3Fef2A25C2622cD0d0b57690479D7350c232038C',
  'amount': 254663701816412642675,
  'shares': 239510421828961408741},
 {'blockNumber': 25098116,
  'timeStamp': 1674667825,
  'gasPrice': 5000000000,
  'gasUsed': 342304,
  'logIndex': 126,
  'transactionIndex': '0x55',
  'event_name': 'Harvest',
  'sender': '0xc77c0e10421e3dd5d50Bfb167630F5215D8AA289',
  'amount': 7651316800067129150},
 {'blockNumber': 25098116,
  'timeStamp': 1674667825,
  'gasPrice': 5000000000,
  'gasUsed': 342304,
  'logIndex': 129,
  'transactionIndex': '0x55',
  '

In [8]:
import pandas as pd


df = pd.DataFrame(decoded_logs)

In [9]:
df

,blockNumber,timeStamp,gasPrice,gasUsed,logIndex,transactionIndex,event_name,sender,amount,shares,lockedAmount,lockedDuration,blockTimestamp,duration,lastDepositedTime
0,25098115,1674667822,5000000000,203064,124,0x47,Harvest,0x3Fef2A25C2622cD0d0b57690479D7350c232038C,168328969601476841510,NaN,NaN,NaN,NaN,NaN,NaN
1,25098115,1674667822,5000000000,203064,128,0x47,Withdraw,0x3Fef2A25C2622cD0d0b57690479D7350c232038C,254663701816412642675,239510421828961408741,NaN,NaN,NaN,NaN,NaN
2,25098116,1674667825,5000000000,342304,126,0x55,Harvest,0xc77c0e10421e3dd5d50Bfb167630F5215D8AA289,7651316800067129150,NaN,NaN,NaN,NaN,NaN,NaN
3,25098116,1674667825,5000000000,342304,129,0x55,Lock,0xc77c0e10421e3dd5d50Bfb167630F5215D8AA289,NaN,141623524832442103351,31140543422695652212,6048000.0,1.674668e+09,NaN,NaN
4,25098116,1674667825,5000000000,342304,131,0x55,Deposit,0xc77c0e10421e3dd5d50Bfb167630F5215D8AA289,31140543422695652212,141623524832442103351,NaN,NaN,NaN,6048000.0,1.674668e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139970,26098102,1677706979,5000000000,372328,129,0x30,Harvest,0xc568793Aa6e769eFed7F553858f79211203Bc234,246098667226544247740,NaN,NaN,NaN,NaN,NaN,NaN
139971,26098102,1677706979,5000000000,372328,132,0x30,Lock,0xc568793Aa6e769eFed7F553858f79211203Bc234,NaN,2224095055057499267847,112852689459998403232,31536000.0,1.677707e+09,NaN,NaN
139972,26098102,1677706979,5000000000,372328,134,0x30,Deposit,0xc568793Aa6e769eFed7F553858f79211203Bc234,112852689459998403232,2224095055057499267847,NaN,NaN,NaN,31536000.0,1.677707e+09
139973,26098112,1677707009,5000000000,247408,148,0x3f,Harvest,0xb08bdc2179Cd1D3622810Fd066d231Bb165aD73C,66513153304471418310,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Save to CSV
df.to_csv("pancake_data.csv", index=False)